# 1. Análisis Exploratorio Inicial de Datos Educativos

Este notebook tiene como objetivo realizar un análisis exploratorio inicial sobre el dataset de estadísticas educativas. Utilizaremos las funciones modularizadas en la carpeta `src/` para la carga, limpieza básica y resumen de los datos.

## 1.1 Configuración del Entorno e Importación de Módulos

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys
import os
    
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
sys.path.append(os.path.abspath(os.path.join('..', 'config')))
    
# Importar funciones y variables
from data_loader import load_data, save_processed_data
from utils import (
    summarize_dataframe,  
    analyze_completeness,
    count_zeros_by_column,
    find_duplicate_columns
)
from settings import (
    RAW_DATA_PATH, 
    PROCESSED_DATA_PATH, 
    COLUMNS_TO_CONVERT_TO_NUMERIC, 
    FIGURES_PATH, 
    COLUMNS_TO_DELETE,
    FILTER_COLUMN,
    FILTER_VALUE
)
# Importamos la función de limpieza 
from data_cleaner import (
    clean_column_names,
    delete_columns,
    handle_missing_values,
    normalize_column_names,
    filter_remove_rows
)  

## 1.2 Carga y Preparación Inicial del Dataset

In [2]:
# Cargar el dataset utilizando la función modular de data_loader.py

df = load_data(RAW_DATA_PATH)

if df is not None:
    # Crear una copia 
    df_work = df.copy()
    print("\nDataFrame original cargado y copia de trabajo creada.")
else:
    df_work = None
    print("No se pudo cargar el DataFrame, df_work es None.")

Intentando cargar el dataset desde: c:\Users\LENOVO\OneDrive\Documentos\python class\Dathon_Academic_MEN\data\raw\MEN_ESTADISTICAS_EN_EDUCACION.csv
Dataset cargado exitosamente. Dimensiones: (14585, 41)

DataFrame original cargado y copia de trabajo creada.


### 1.2.1 Resumen Inicial del DataFrame

In [3]:
# Resumen del DataFrame
summarize_dataframe(df_work)

El DataFrame tiene 14585 filas y 41 columnas.

Primeras filas del DataFrame:
    AÑO  CÓDIGO_MUNICIPIO   MUNICIPIO  CÓDIGO_DEPARTAMENTO DEPARTAMENTO  \
0  2023              5001    Medellín                    5    Antioquia   
1  2023              5002   Abejorral                    5    Antioquia   
2  2023              5004    Abriaquí                    5    Antioquia   
3  2023              5021  Alejandría                    5    Antioquia   
4  2023              5030       Amagá                    5    Antioquia   

   CÓDIGO_ETC              ETC POBLACIÓN_5_16  TASA_MATRICULACIÓN_5_16  \
0      3759.0         Medellín         377562                    96.15   
1      3758.0  Antioquia (ETC)           3634                    74.38   
2      3758.0  Antioquia (ETC)            503                    62.62   
3      3758.0  Antioquia (ETC)            864                    81.37   
4      3758.0  Antioquia (ETC)           5060                    78.30   

   COBERTURA_NETA  ...  REP

### 1.2.2 Análisis de valores nulos

In [4]:
analyze_completeness(df_work)

Total de valores posibles: 597985
Total de valores no nulos: 579275
Porcentaje de completitud del DataFrame: 96.87%

Porcentaje de valores nulos por columna (orden descendente):
TAMAÑO_PROMEDIO_DE_GRUPO       48.083648
SEDES_CONECTADAS_A_INTERNET    46.739801
DESERCIÓN_TRANSICIÓN            6.191292
DESERCIÓN_MEDIA                 5.032568
DESERCIÓN_SECUNDARIA            1.851217
DESERCIÓN_PRIMARIA              1.659239
REPITENCIA_TRANSICIÓN           1.090161
REPITENCIA_SECUNDARIA           1.042167
REPITENCIA_PRIMARIA             1.014741
REPROBACIÓN_MEDIA               0.994172
REPITENCIA                      0.980459
DESERCIÓN                       0.973603
REPITENCIA_MEDIA                0.953034
COBERTURA_BRUTA_MEDIA           0.870758
TASA_MATRICULACIÓN_5_16         0.788481
COBERTURA_NETA                  0.761056
REPROBACIÓN_SECUNDARIA          0.726774
APROBACIÓN_MEDIA                0.692492
REPROBACIÓN_PRIMARIA            0.665067
COBERTURA_BRUTA_TRANSICIÓN      0.665067
CO

Justificación para eliminar variables

Las variables **TAMAÑO_PROMEDIO_DE_GRUPO** y **SEDES_CONECTADAS_A_INTERNET** presentan un alto porcentaje de datos faltantes, con un **48.08%** y **46.74%** de valores nulos, respectivamente. 

Este nivel de incompletitud limita la calidad y confiabilidad de los análisis, especialmente en modelos predictivos o análisis globales que requieren datos consistentes.

Por esta razón, se decidió eliminar estas variables del análisis principal para garantizar resultados más robustos y representativos. 

Sin embargo, se podrán considerar análisis focalizados en subconjuntos donde estas variables estén completas para explorar su posible impacto en la calidad educativa.

### 1.2.3 Análisis de columnas con Ceros

In [5]:
columns_with_zeros = count_zeros_by_column(df_work)

Número de columnas con al menos un valor en cero: 35

Columnas con valores en cero y su cantidad:
CÓDIGO_MUNICIPIO                   3
CÓDIGO_DEPARTAMENTO                3
CÓDIGO_ETC                         3
TASA_MATRICULACIÓN_5_16            5
COBERTURA_NETA                     8
COBERTURA_NETA_TRANSICIÓN          9
COBERTURA_NETA_PRIMARIA            8
COBERTURA_NETA_SECUNDARIA         26
COBERTURA_NETA_MEDIA              50
COBERTURA_BRUTA                    8
COBERTURA_BRUTA_TRANSICIÓN         8
COBERTURA_BRUTA_PRIMARIA           8
COBERTURA_BRUTA_SECUNDARIA        26
COBERTURA_BRUTA_MEDIA             43
SEDES_CONECTADAS_A_INTERNET      284
DESERCIÓN                         55
DESERCIÓN_TRANSICIÓN            1219
DESERCIÓN_PRIMARIA               355
DESERCIÓN_SECUNDARIA             195
DESERCIÓN_MEDIA                  698
APROBACIÓN                         3
APROBACIÓN_TRANSICIÓN          10330
APROBACIÓN_PRIMARIA                3
APROBACIÓN_SECUNDARIA             14
APROBACIÓN_MED

### 1.2.4 Análisis de Columnas Duplicadas en valores

In [6]:
duplicates = find_duplicate_columns(df_work)

Las columnas 'APROBACIÓN_TRANSICIÓN' y 'REPROBACIÓN_TRANSICIÓN' tienen los mismos datos.


# 2: Limpieza

### 2.1 Limpia y estandariza los nombres de las columnas 

In [7]:
df_cleaned = clean_column_names(df_work)

Columnas estandarizadas ✅.


### 2.2 Eliminar columnas

In [8]:
df_delete = delete_columns(df_cleaned, COLUMNS_TO_DELETE )

Columnas Eliminadas. ✅


### 2.3 Imputa ceros y valores nulos en columnas numéricas

In [9]:
df_imp = handle_missing_values(df_delete)

✅ Imputación completa.
Número de ceros por columna después de imputar:  0 

Número de nulos por columna después de imputar:  0


### 2.4 Imputar valores en columnas categóricas (texto)

In [10]:
df_procesado = normalize_column_names(
    df_imp,
    columns_to_convert_to_numeric=COLUMNS_TO_CONVERT_TO_NUMERIC,
    imputar=True
)


 Valores nulos en columnas categóricas:
poblacion_5_16    6 

✅ Columna 'poblacion_5_16' convertida a numérico
   - Valores nulos imputados: 6 → 0
   - Mediana usada para imputación: 2640.00
   - Tipo de dato actual: float64



### 2.5 Eliminar filas especificas

In [11]:
df_processed = filter_remove_rows(
    df_procesado,
    FILTER_COLUMN,
    FILTER_VALUE    
)

✅ Se eliminaron 3 filas donde 'municipio' == NACIONAL


### 2.6 Guardar el DataFrame limpio para uso futuro

In [12]:
save_processed_data(df_processed, PROCESSED_DATA_PATH)

Datos procesados guardados exitosamente en: c:\Users\LENOVO\OneDrive\Documentos\python class\Dathon_Academic_MEN\data\processed\datos_educacion_limpios.csv
